In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
import missingno as msn
import datetime
from pandas_profiling import ProfileReport
import sklearn

In [20]:
df = pd.read_parquet("escooter_history_2022.parquet")



In [21]:
df.loc[df.temp > 50, "temp"] = df.temp.div(100)

In [22]:
df = pd.get_dummies(df)

In [23]:
df

,datetime,holiday,workingday,temp,atemp,humidity,windspeed,registered_customer,"weather_clear, few clouds","weather_cloudy, mist",weather_heacy rain or thunderstorm or snow or ice pallets,weather_light snow or rain or thunderstorm
0,2020-01-04 00:00:09,0.0,0.0,9.84,14.395,81.0,0.0000,True,1,0,0,0
1,2020-01-04 00:00:41,0.0,0.0,9.84,14.395,81.0,0.0000,True,1,0,0,0
2,2020-01-04 00:01:20,0.0,0.0,9.84,14.395,81.0,0.0000,True,1,0,0,0
3,2020-01-04 00:04:12,0.0,0.0,9.84,14.395,81.0,0.0000,True,1,0,0,0
4,2020-01-04 00:15:19,0.0,0.0,9.84,14.395,81.0,0.0000,True,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
3755410,2022-01-03 23:58:10,0.0,1.0,10.66,13.635,65.0,8.9981,True,1,0,0,0
3755411,2022-01-03 23:58:28,0.0,1.0,10.66,13.635,65.0,8.9981,True,1,0,0,0
3755412,2022-01-03 23:58:46,0.0,1.0,10.66,13.635,65.0,8.9981,True,1,0,0,0
3755413,2022-01-03 23:59:16,0.0,1.0,10.66,13.635,65.0,8.9981,True,1,0,0,0


In [24]:
df_2 = df.groupby(pd.Grouper(key="datetime",freq="D")).agg(Anzahl_pro_Tag=("registered_customer","count"),temp=("temp","mean"),humidity=("humidity","mean"),windspeed=("windspeed","mean"),workingday=("workingday","mean"),holiday=("holiday","mean"),Anzahl_registered=("registered_customer","sum"),clear_few_clouds=("weather_clear, few clouds","mean"),cloudy_mist=("weather_cloudy, mist","mean"),heacy_rain_thunderstorm_snow_icepallets=("weather_heacy rain or thunderstorm or snow or ice pallets","mean"),ligth_snow_rain_thunderstorm=("weather_light snow or rain or thunderstorm","mean")).reset_index()

In [25]:
df_2.reset_index(drop=True)

,datetime,Anzahl_pro_Tag,temp,humidity,windspeed,workingday,holiday,Anzahl_registered,clear_few_clouds,cloudy_mist,heacy_rain_thunderstorm_snow_icepallets,ligth_snow_rain_thunderstorm
0,2020-01-04,985,16.489909,79.678173,16.226271,0.00000,0.0,654,0.309645,0.617259,0.0,0.073096
1,2020-01-05,801,14.415006,67.141074,15.875136,0.00000,0.0,670,0.205993,0.520599,0.0,0.273408
2,2020-01-06,1349,8.440104,40.287620,14.724723,1.00000,0.0,1229,1.000000,0.000000,0.0,0.000000
3,2020-01-07,1562,9.051498,56.411012,12.351703,1.00000,0.0,1454,0.973111,0.026889,0.0,0.000000
4,2020-01-08,1600,9.649350,38.230000,14.113182,1.00000,0.0,1518,1.000000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
726,2021-12-30,4231,10.475495,59.472229,22.242182,1.00000,0.0,4024,0.540298,0.454266,0.0,0.005436
727,2021-12-31,8144,10.419155,58.653487,10.341263,1.00000,0.0,7629,0.328217,0.671783,0.0,0.000000
728,2022-01-01,5932,10.551763,76.322151,8.237662,0.00000,0.0,5293,0.210890,0.630479,0.0,0.158631
729,2022-01-02,5182,10.778680,46.243342,23.974396,0.00000,0.0,4392,0.761289,0.238711,0.0,0.000000


In [20]:
df_2.dtypes

datetime             datetime64[ns]
Anzahl_pro_Tag                int64
temp                        float64
humidity                    float64
windspeed                   float64
workingday                  float64
holiday                     float64
Anzahl_registered             Int64
Anteil_registered           Float64
weekend                        bool
date_delta                    int64
dtype: object

In [26]:
df_2["workingday"] = df_2["workingday"].round()

In [27]:

df_2["holiday"] = df_2["holiday"].round() 

In [28]:
df_2["Anteil_registered"] = df_2["Anzahl_registered"]/df_2["Anzahl_pro_Tag"]

In [29]:
df_2["weekend"] = df_2["datetime"].dt.weekday > 4

In [30]:
df_2['date_delta'] = (df_2['datetime'] - df_2['datetime'].min())  
df_2["date_delta"] = df_2.date_delta.dt.days

In [31]:
df_2.drop("Anzahl_registered",axis=1)

,datetime,Anzahl_pro_Tag,temp,humidity,windspeed,workingday,holiday,clear_few_clouds,cloudy_mist,heacy_rain_thunderstorm_snow_icepallets,ligth_snow_rain_thunderstorm,Anteil_registered,weekend,date_delta
0,2020-01-04,985,16.489909,79.678173,16.226271,0.0,0.0,0.309645,0.617259,0.0,0.073096,0.663959,True,0
1,2020-01-05,801,14.415006,67.141074,15.875136,0.0,0.0,0.205993,0.520599,0.0,0.273408,0.836454,True,1
2,2020-01-06,1349,8.440104,40.287620,14.724723,1.0,0.0,1.000000,0.000000,0.0,0.000000,0.911045,False,2
3,2020-01-07,1562,9.051498,56.411012,12.351703,1.0,0.0,0.973111,0.026889,0.0,0.000000,0.930858,False,3
4,2020-01-08,1600,9.649350,38.230000,14.113182,1.0,0.0,1.000000,0.000000,0.0,0.000000,0.94875,False,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,2021-12-30,4231,10.475495,59.472229,22.242182,1.0,0.0,0.540298,0.454266,0.0,0.005436,0.951075,False,726
727,2021-12-31,8144,10.419155,58.653487,10.341263,1.0,0.0,0.328217,0.671783,0.0,0.000000,0.936763,False,727
728,2022-01-01,5932,10.551763,76.322151,8.237662,0.0,0.0,0.210890,0.630479,0.0,0.158631,0.892279,True,728
729,2022-01-02,5182,10.778680,46.243342,23.974396,0.0,0.0,0.761289,0.238711,0.0,0.000000,0.847549,True,729


In [32]:
#trainings-Test-Split
features = ["date_delta", 'temp', 'humidity','windspeed',"workingday","holiday","Anteil_registered","weekend","clear_few_clouds","cloudy_mist","heacy_rain_thunderstorm_snow_icepallets","ligth_snow_rain_thunderstorm"] # Liste von Features
X = df_2[features]
y = df_2.Anzahl_pro_Tag

In [33]:
X

,date_delta,temp,humidity,windspeed,workingday,holiday,Anteil_registered,weekend,clear_few_clouds,cloudy_mist,heacy_rain_thunderstorm_snow_icepallets,ligth_snow_rain_thunderstorm
0,0,16.489909,79.678173,16.226271,0.0,0.0,0.663959,True,0.309645,0.617259,0.0,0.073096
1,1,14.415006,67.141074,15.875136,0.0,0.0,0.836454,True,0.205993,0.520599,0.0,0.273408
2,2,8.440104,40.287620,14.724723,1.0,0.0,0.911045,False,1.000000,0.000000,0.0,0.000000
3,3,9.051498,56.411012,12.351703,1.0,0.0,0.930858,False,0.973111,0.026889,0.0,0.000000
4,4,9.649350,38.230000,14.113182,1.0,0.0,0.94875,False,1.000000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
726,726,10.475495,59.472229,22.242182,1.0,0.0,0.951075,False,0.540298,0.454266,0.0,0.005436
727,727,10.419155,58.653487,10.341263,1.0,0.0,0.936763,False,0.328217,0.671783,0.0,0.000000
728,728,10.551763,76.322151,8.237662,0.0,0.0,0.892279,True,0.210890,0.630479,0.0,0.158631
729,729,10.778680,46.243342,23.974396,0.0,0.0,0.847549,True,0.761289,0.238711,0.0,0.000000


In [13]:
from sklearn import linear_model
from sklearn.model_selection import train_test_split

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

In [35]:
from sklearn.preprocessing import StandardScaler
#Standardisierung

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [36]:
#Training

model_lr = linear_model.LinearRegression()
model_lr.fit(X_train, y_train)
y_train_pred = model_lr.predict(X_train)
y_test_pred = model_lr.predict(X_test)

In [37]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


In [38]:
r2_test = r2_score(y_test, y_test_pred)
mse_test = mean_squared_error(y_test, y_test_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)

In [40]:
print(r2_test)

0.649932633731239


In [41]:
print(mse_test)

2144129.392634709


In [42]:
print(mae_test)

1030.9445395844177


problem cannot turn datetime into float